# ORC(Optical Character Recognition)
## 手写数字图片的OCR

In [28]:
import numpy as np
import cv2 as cv
np.set_printoptions(threshold=np.inf)

img = cv.imread('../data/digits.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

# 现在我们将图像分割为5000个单元格，每个单元格为20x20
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]
# 使其成为一个Numpy数组。它的大小将是（50,100,20,20）
x = np.array(cells)

# 现在我们准备train_data和test_data。取出前50％的数据用于训练，其余50％用于测试。
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# 为训练和测试数据创建标签
#这里我理解为digits图片中有10个数字
# 共是2500个 每组250个 所以是重复250次
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis] # Size = (2500,1)

test_labels = train_labels.copy()
# 初始化kNN，训练数据，然后使用k = 1的测试数据对其进行测试
knn = cv.ml.KNearest_create()

# 训练集是左边的50％像素2500x400,标签是0-9的重复
# 也就是每个字对应的400像素为一组样本,标签是1-9的重复(250次)
# 图片像素的读取是从左到右,从上到下的顺序吗?因为reshape(-1,400)是按行读取的
knn.train(train, cv.ml.ROW_SAMPLE, train_labels)

# 拿右边的50％像素2500x400作为测试集
ret,result,neighbours,dist = knn.findNearest(test,k=5)

# 现在，我们检查分类的准确性
# 为此，将结果与test_labels进行比较，并检查哪个错误
matches = (result==test_labels)    # 其实是这样的
correct = np.count_nonzero(matches)#计算非零元素的个数
accuracy = correct*100.0/result.size#计算准确率

print(accuracy)

# 保存数据
np.savez('knn_data.npz',train=train, train_labels=train_labels)
# 现在加载数据
with np.load('knn_data.npz') as data:
    train = data['train']
    train_labels = data['train_labels']


91.76


## 英文字母的OCR

In [32]:
import cv2 as cv
import numpy as np
# 加载数据，转换器将字母转换为数字 按照aicll码解析
data= np.loadtxt('../data/letter-recognition.data', dtype= 'float32', delimiter = ',',
                    converters= {0: lambda ch: ord(ch)-ord('A')})

# 将数据分为两个，每个10000个以进行训练和测试
train, test = np.vsplit(data,2)

# 将训练数据和测试数据拆分为特征和响应
# 将第一列分离开来,作为标签,剩下的作为特征
responses, trainData = np.hsplit(train,[1])
labels, testData = np.hsplit(test,[1])

# 初始化kNN, 分类, 测量准确性
knn = cv.ml.KNearest_create()
knn.train(trainData, cv.ml.ROW_SAMPLE, responses)
ret, result, neighbours, dist = knn.findNearest(testData, k=5)

print( "ret:  {}\n".format(ret))
print( "result:  {}\n".format(result) )
print( "neighbours:  {}\n".format(neighbours) )
print( "distance:  {}\n".format(dist) )

correct = np.count_nonzero(result == labels)
accuracy = correct*100.0/10000
print( accuracy )


ret:  22.0

result:  [[22.]
 [ 9.]
 [ 3.]
 [14.]
 [16.]
 [ 7.]
 [17.]
 [19.]
 [20.]
 [10.]
 [11.]
 [ 6.]
 [23.]
 [24.]
 [ 8.]
 [13.]
 [24.]
 [12.]
 [ 4.]
 [12.]
 [ 0.]
 [17.]
 [17.]
 [ 8.]
 [ 0.]
 [25.]
 [18.]
 [ 2.]
 [17.]
 [ 3.]
 [11.]
 [21.]
 [15.]
 [ 4.]
 [ 7.]
 [20.]
 [25.]
 [18.]
 [ 8.]
 [14.]
 [ 2.]
 [23.]
 [13.]
 [ 8.]
 [14.]
 [14.]
 [11.]
 [21.]
 [ 6.]
 [12.]
 [13.]
 [18.]
 [25.]
 [ 1.]
 [ 9.]
 [18.]
 [ 3.]
 [14.]
 [ 4.]
 [15.]
 [20.]
 [23.]
 [ 0.]
 [16.]
 [16.]
 [ 6.]
 [11.]
 [10.]
 [ 4.]
 [ 9.]
 [ 4.]
 [22.]
 [15.]
 [ 6.]
 [17.]
 [15.]
 [23.]
 [16.]
 [ 8.]
 [ 9.]
 [ 1.]
 [ 3.]
 [ 1.]
 [25.]
 [ 3.]
 [ 1.]
 [ 5.]
 [ 2.]
 [13.]
 [ 3.]
 [17.]
 [22.]
 [ 1.]
 [ 8.]
 [ 4.]
 [ 2.]
 [ 4.]
 [23.]
 [20.]
 [11.]
 [12.]
 [10.]
 [ 5.]
 [24.]
 [ 6.]
 [19.]
 [ 8.]
 [ 4.]
 [19.]
 [ 8.]
 [19.]
 [21.]
 [ 3.]
 [ 7.]
 [13.]
 [19.]
 [ 3.]
 [14.]
 [17.]
 [22.]
 [16.]
 [22.]
 [13.]
 [ 2.]
 [23.]
 [ 1.]
 [ 8.]
 [10.]
 [11.]
 [21.]
 [12.]
 [21.]
 [19.]
 [ 2.]
 [11.]
 [ 6.]
 [25.]
 [18.]
 [21.]
 [23.]